In [38]:
import gzip
from collections import defaultdict
from sklearn import linear_model
import csv
import heapq
import string
from nltk.corpus import stopwords

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [7]:
def parseData(fname):
    for l in gzip.open(fname):
        d = eval(l)
        yield d

In [8]:
data = list(parseData("CA_5.json.gz"))

In [9]:
places = list(parseData('places_CA.json.gz'))

In [10]:
reviews = list(parseData('reviews_CA.json.gz'))

In [26]:
data[0]

{'rating': 4.0,
 'reviewerName': 'Mary Gainza',
 'reviewText': 'Gap always has a jean that fits awesomely, i wish their prices were a bit competitive in comparison to their outlets, also the amount of the pieces they have in store sometimes fly so quick that is hard to find an specific size with their best sellers, but in general i really like this store and most of their products.',
 'categories': ['Clothing Store',
  "Women's Clothing Store",
  "Children's Clothing Store"],
 'gPlusPlaceId': '100556368174926958612',
 'unixReviewTime': 1355436757,
 'reviewTime': 'Dec 13, 2012',
 'gPlusUserId': '100000715097692381911'}

In [22]:
places[9]

{'name': 'Nail Perfection',
 'price': None,
 'address': ['556 Las Posas Rd', 'Camarillo, CA 93010'],
 'hours': None,
 'phone': '(805) 987-0992',
 'closed': False,
 'gPlusPlaceId': '100068746766818502566',
 'gps': [34.218397, -119.069823]}

In [13]:
reviews[0]

{'rating': 4.0,
 'reviewerName': 'william spindler',
 'reviewText': 'Best War Wanton soup in Red Bluff',
 'categories': ['Asian Restaurant', 'Chinese Restaurant'],
 'gPlusPlaceId': '106591714648856494903',
 'unixReviewTime': 1394669496,
 'reviewTime': 'Mar 12, 2014',
 'gPlusUserId': '100000032416892623125'}

In [ ]:
wordCount = defaultdict(int)
for d in data:
    if type(d['reviewText']) == str:
        for w in d['reviewText'].split():
            wordCount[w] += 1
wordCount

In [34]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    if type(d['reviewText']) == str:
        r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
        for w in r.split():
            wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [ ]:
wordCount

In [36]:
words = [x[1] for x in counts[:1000]]

In [ ]:
words

In [40]:
#removing stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mgvasque/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [44]:
stop_words = set(stopwords.words('english'))

In [ ]:
stop_words

In [47]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    if type(d['reviewText']) == str:
        r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
        for w in r.split():
            if w not in stop_words:
                wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [51]:
#EDA top words
counts[:10]

[(28827, 'food'),
 (28762, 'good'),
 (27991, 'great'),
 (23042, 'place'),
 (14069, 'service'),
 (11499, 'like'),
 (11397, 'get'),
 (10658, 'best'),
 (10246, 'one'),
 (10117, 'really')]

In [ ]:
#sentimental analysis

In [59]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

In [ ]:
def feature(d):
    feat = [0]*len(words)
    if type(d['reviewText']) == str:
        r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
        ws = r.split()
        ws2 = [' '.join(x) for x in list(zip(ws[:-1],ws[1:]))]
        ws3 = [' '.join(x) for x in list(zip(ws[:-2],ws[1:-1],ws[2:]))]
        ws4 = [' '.join(x) for x in list(zip(ws[:-3],ws[1:-2],ws[2:-1],ws[3:]))]
        ws5 = [' '.join(x) for x in list(zip(ws[:-4],ws[1:-3],ws[2:-2],ws[3:-1],ws[4:]))]
        for w in ws + ws2 + ws3 + ws4 + ws5:
            if w in words:
                feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

In [ ]:
X = [feature(d) for d in data]
y = [d['rating'] for d in data]